### Import necessary libs

In [1]:
from ultralytics import YOLO
import os
import colored
import shutil
import random
from pathlib import Path
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np
from datetime import datetime
import pandas as pd
import yaml

### Configuration parametrs

In [2]:
DATA_YAML = "/home/kronbii/datasets/fortnite/lunar/data.yaml"
TRAIN_DIR = "/home/kronbii/datasets/fortnite/lunar/train"
TEST_DIR = "/home/kronbii/datasets/fortnite/lunar/test"
VAL_DIR = "/home/kronbii/datasets/fortnite/lunar/valid"
NAMES = ['person']


PRETRAINED_MODEL = "yolov8n.pt"  
PROJECT_NAME = "person_detection"
EXPERIMENT_NAME = "lunar-v8-960"
EPOCHS = 50
BATCH_SIZE = 8
IMAGE_SIZE = 960

### Fine tuning main function

In [3]:
def fine_tune_YOLO():
    """
    Fine-tune YOLOv8 model to detect game characters as 'person' class
    This leverages pretrained knowledge for faster, better training
    """
    
    print("=" * 60)
    print("[INFO] YOLOv8 Person Detection Fine-tuning (Game Characters)")
    print("=" * 60)
    
    # Check if dataset exists
    if not os.path.exists(DATA_YAML):
        print(colored.fg("red") + f"[ERROR] Dataset file not found: {DATA_YAML}" + colored.attr("reset"))
        return
    
    # Load pretrained model
    print(f"[INFO] Loading pretrained model: {PRETRAINED_MODEL}")
    model = YOLO(PRETRAINED_MODEL)
    
    # Display model info
    print(f"[MODEL] Model architecture: {PRETRAINED_MODEL}")
    print(f"[MODEL] Dataset: {DATA_YAML}")
    print(f"[MODEL] Epochs: {EPOCHS}")
    print(f"[MODEL] Image size: {IMAGE_SIZE}")
    print(f"[MODEL] Batch size: {BATCH_SIZE}")
    
    # Train the model with transfer learning settings
    results = model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        imgsz=IMAGE_SIZE,
        batch=BATCH_SIZE,
        project=PROJECT_NAME,
        name=EXPERIMENT_NAME,
        save=True,
        save_period=5,   
        patience=25,     
        device=0,        # Use GPU 0, 'cpu' if no GPU
        workers=8,
        cache=True,
        # Optimized settings for fine-tuning person detection
        lr0=0.0007,       # Lower initial learning rate for fine-tuning
        lrf=0.05,         # Learning rate final multiplier
        warmup_epochs=5, # Warmup epochs
		# Minimal augmentation for stability
		hsv_h=0.005,      # Very minimal hue shift
		hsv_s=0.6,        # Reduced saturation
		hsv_v=0.6,        # Reduced value changes
		degrees=5.0,      # Very small rotation
		translate=0.1,    # Minimal translation
		scale=0.2,        # Minimal scale changes
		fliplr=0.5,       # Reduced horizontal flip
		mosaic=0.5,       # Significantly reduced mosaic
		mixup=0.1,        # Very light mixup
    )
    
    print("\n[INFO] Fine-tuning completed!")
    print(f"[INFO] Results saved to: {PROJECT_NAME}/{EXPERIMENT_NAME}")
    
    # Validate
    metrics = model.val()
    
    return model, results, metrics

### Dataset structure validator

In [4]:
def verify_dataset_structure():
    # Check if data.yaml exists and create if needed
    if not os.path.exists(DATA_YAML):
        train_images_path = os.path.join(TRAIN_DIR, "images")
        val_images_path = os.path.join(VAL_DIR, "images")
        test_images_path = os.path.join(TEST_DIR, "images")
        
        yaml_data = {
            'train': train_images_path,
            'val': val_images_path, 
            'test': test_images_path,
            'nc': len(NAMES),
            'names': NAMES
        }
        
        with open(DATA_YAML, 'w') as f:
            yaml.dump(yaml_data, f, default_flow_style=False)
    
    # Count and verify images and labels
    train_images_dir = os.path.join(TRAIN_DIR, "images")
    train_labels_dir = os.path.join(TRAIN_DIR, "labels")
    
    train_images = len([f for f in os.listdir(train_images_dir) 
                       if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))])
    train_labels = len([f for f in os.listdir(train_labels_dir) if f.endswith('.txt')])
    
    return train_images > 0 and train_labels > 0

In [5]:
validity = verify_dataset_structure()
if not validity:
    print("[ERROR] INVALID DATASET")
    exit()

In [6]:
print(colored.fg("green") + "[INFO] STARTING SUCCESS" + colored.attr("reset"))
model, results, metrics = fine_tune_YOLO()

[INFO] STARTING SUCCESS
[INFO] YOLOv8 Person Detection Fine-tuning (Game Characters)
[INFO] Loading pretrained model: yolov8n.pt
[MODEL] Model architecture: yolov8n.pt
[MODEL] Dataset: /home/kronbii/datasets/fortnite/lunar/data.yaml
[MODEL] Epochs: 50
[MODEL] Image size: 960
[MODEL] Batch size: 8
New https://pypi.org/project/ultralytics/8.3.204 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.179 🚀 Python-3.9.23 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7815MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/kronbii/datasets/fortnite/lunar/data.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, f

train: Scanning /home/kronbii/datasets/fortnite/lunar/train/labels.cache... 426 images, 84 backgrounds, 0 corrupt: 100%|██████████| 426/426 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (1.1GB RAM): 100%|██████████| 426/426 [00:00<00:00, 1421.24it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 102.1±38.1 MB/s, size: 36.3 KB)


...

                   all         50         44      0.806      0.565      0.706      0.329
Speed: 0.6ms preprocess, 4.6ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to person_detection/lunar-v8-960

[INFO] Fine-tuning completed!
[INFO] Results saved to: person_detection/lunar-v8-960
Ultralytics 8.3.179 🚀 Python-3.9.23 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 7815MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 130.6±87.0 MB/s, size: 34.3 KB)


val: Scanning /home/kronbii/datasets/fortnite/lunar/valid/labels.cache... 50 images, 7 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (0.1GB RAM): 100%|██████████| 50/50 [00:00<00:00, 1063.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  5.24it/s]


                   all         50         44      0.806      0.565      0.705      0.328
Speed: 5.2ms preprocess, 12.2ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to person_detection/lunar-v8-9602


In [8]:
if model is not None:
	print("\n" + "=" * 20)
	print(colored.fg("green") + "[INFO] TRAINING SUCCESS" + colored.attr("reset"))
	print("=" * 20)
	
	if metrics:
		print("\n" + "=" * 20)
		print(colored.fg("green") + "[INFO] EVALUATION COMPLETE" + colored.attr("reset"))
		print("=" * 20)
		
		print(f"\n FINAL PERFORMANCE SUMMARY:")
		print("-" * 40)
		print(colored.fg("blue") + f"[METRIC] mAP@0.5: {metrics.results_dict['metrics/mAP50(B)']:.3f}" + colored.attr("reset"))
		print(colored.fg("blue") + f"[METRIC] Precision: {metrics.results_dict['metrics/precision(B)']:.3f}" + colored.attr("reset"))
		print(colored.fg("blue") + f"[METRIC] Recall: {metrics.results_dict['metrics/recall(B)']:.3f}" + colored.attr("reset"))
		print(colored.fg("blue") + f"[METRIC] mAP@0.5:0.95: {metrics.results_dict['metrics/mAP50-95(B)']:.3f}" + colored.attr("reset"))
  
		# Save metrics to a txt file
		metrics_file = os.path.join(PROJECT_NAME, EXPERIMENT_NAME, "final_metrics.txt")
		with open(metrics_file, "w") as f:
			f.write("FINAL PERFORMANCE SUMMARY:\n")
			f.write("-" * 40 + "\n")
			f.write(f"mAP@0.5: {metrics.results_dict['metrics/mAP50(B)']:.3f}\n")
			f.write(f"Precision: {metrics.results_dict['metrics/precision(B)']:.3f}\n")
			f.write(f"Recall: {metrics.results_dict['metrics/recall(B)']:.3f}\n")
			f.write(f"mAP@0.5:0.95: {metrics.results_dict['metrics/mAP50-95(B)']:.3f}\n")
		print(colored.fg("yellow") + f"[INFO] Metrics saved to {metrics_file}" + colored.attr("reset"))

	else:
		print(colored.fg("red") + "[ERROR] Training failed!" + colored.attr("reset"))


[INFO] TRAINING SUCCESS

[INFO] EVALUATION COMPLETE

 FINAL PERFORMANCE SUMMARY:
----------------------------------------
[METRIC] mAP@0.5: 0.705
[METRIC] Precision: 0.806
[METRIC] Recall: 0.565
[METRIC] mAP@0.5:0.95: 0.328
[INFO] Metrics saved to person_detection/lunar-v8-960/final_metrics.txt
